<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#7-1-一歩ずつ安全に" data-toc-modified-id="7-1-一歩ずつ安全に-1">7-1 一歩ずつ安全に</a></span></li><li><span><a href="#7-3-Huffmanのアルゴリズム" data-toc-modified-id="7-3-Huffmanのアルゴリズム-2">7-3 Huffmanのアルゴリズム</a></span><ul class="toc-item"><li><span><a href="#リスト7-1-Huffmanのアルゴリズム" data-toc-modified-id="リスト7-1-Huffmanのアルゴリズム-2.1">リスト7-1 Huffmanのアルゴリズム</a></span></li><li><span><a href="#リスト7-2-Huffman木からHuffman符号を抽出" data-toc-modified-id="リスト7-2-Huffman木からHuffman符号を抽出-2.2">リスト7-2 Huffman木からHuffman符号を抽出</a></span></li></ul></li><li><span><a href="#7-4-最小全域木" data-toc-modified-id="7-4-最小全域木-3">7-4 最小全域木</a></span><ul class="toc-item"><li><span><a href="#Kruskalのアルゴリズム" data-toc-modified-id="Kruskalのアルゴリズム-3.1">Kruskalのアルゴリズム</a></span></li><li><span><a href="#リスト7-3-Kruskalのアルゴリズムの簡単な実装" data-toc-modified-id="リスト7-3-Kruskalのアルゴリズムの簡単な実装-3.2">リスト7-3 Kruskalのアルゴリズムの簡単な実装</a></span></li><li><span><a href="#リスト7-4-Kruskalのアルゴリズム" data-toc-modified-id="リスト7-4-Kruskalのアルゴリズム-3.3">リスト7-4 Kruskalのアルゴリズム</a></span></li><li><span><a href="#Primのアルゴリズム" data-toc-modified-id="Primのアルゴリズム-3.4">Primのアルゴリズム</a></span></li><li><span><a href="#リスト7-5-Primのアルゴリズム" data-toc-modified-id="リスト7-5-Primのアルゴリズム-3.5">リスト7-5 Primのアルゴリズム</a></span></li></ul></li></ul></div>

# 7-1 一歩ずつ安全に
貪慾法のシンプルな問題例として、お釣りを用意すること、つまり、できるだけ少ないコインとお札で与えられた合計金額に近い値を作る問題を考えてみましょう。


以下のコードで示すように、そのお金の単位を降順でソートし、最も大きい額面のものから使っていきます。（浮動小数点の問題を避けるために100倍の値のセントで計算しています）。

In [1]:
denom = [10000, 5000, 2000, 1000, 500, 200, 100, 50, 25, 10, 5, 1]
owed = 5632
payed = []
for d in denom:
    while owed >= d:
        owed -= d
        payed.append(d)

sum(payed)
# 5632

5632

In [2]:
payed
[5000, 500, 100, 25, 5, 1, 1]

[5000, 500, 100, 25, 5, 1, 1]

In [3]:
len(payed)
# 7

7

# 7-3 Huffmanのアルゴリズム

## リスト7-1 Huffmanのアルゴリズム

以下は、Huffmanのアルゴリズムの1つの実装例です。木は入れ子構造のリストで表現し、その木の集合（森）を部分解として組み上げています。

In [4]:
from heapq import heapify, heappush, heappop
from itertools import count


def huffman(seq, frq):
    num = count()
    trees = list(zip(frq, num, seq))            # num が有効な順序を保証
    heapify(trees)                              # frqに基づいた最小ヒープ
    while len(trees) > 1:                       # すべてが結合されるまで
        fa, _, a = heappop(trees)               # 最小の木を2つ選ぶ
        fb, _, b = heappop(trees)
        n = next(num)
        heappush(trees, (fa+fb, n, [a, b]))     # 　結合し、再度追加する
    return trees[0][-1]

以下が、このコードの使い方の一例です。

In [5]:
seq = "abcdefghi"
frq = [4, 5, 6, 9, 11, 12, 15, 16, 20]
huffman(seq, frq)
# [['i', [['a', 'b'], 'e']], [['f', 'g'], [['c', 'd'], 'h']]]

[['i', [['a', 'b'], 'e']], [['f', 'g'], [['c', 'd'], 'h']]]

## リスト7-2 Huffman木からHuffman符号を抽出

テキストの圧縮および解凍をするには、前処理と後処理が必要です。最初に、（例えば、`collections`モジュールの`Counter`クラスを使って）頻度を算出するために文字を数える必要があります。次に、Huffman木が得られたら、以下のリスト7-2に示すように、再帰を使った単純な巡回によって、すべての文字コードを見つけます。

In [6]:
def codes(tree, prefix=""):
    if len(tree) == 1:
        yield (tree, prefix)                    # コードと葉
        return
    for bit, child in zip("01", tree):          # 左は(0)、右は (1)を表す
        for pair in codes(child, prefix + bit): # コードを再帰的に取得
            yield pair

In [7]:
C = dict(codes(_))              #  "_" は1つ前のセルの出力を保持しています。
C['i'], C['a'], C['c']
# ('00', '0100', '1100')

('00', '0100', '1100')

# 7-4 最小全域木

## Kruskalのアルゴリズム

## リスト7-3 Kruskalのアルゴリズムの簡単な実装

以下、Kruskalのアルゴリズムをマップ`C`を使って「指向先（pointing）」として実装しています。

In [8]:
def naive_find(C, u):                           # 成分の代表を見つける
    while C[u] != u:                            # 代表は自身を指している場合を除く
        u = C[u]
    return u


def naive_union(C, u, v):
    u = naive_find(C, u)                        # 両方の代表を見つける
    v = naive_find(C, v)
    C[u] = v                                    # 1つの代表がもう1つの代表を指すようにする


def naive_kruskal(G):
    E = [(G[u][v], u, v) for u in G for v in G[u]]
    T = set()                                   # 空の部分解
    C = {u: u for u in G}                       # 成分の代表
    for _, u, v in sorted(E):                   # 重みによってソートされたエッジ
        if naive_find(C, u) != naive_find(C, v):
            T.add((u, v))                       # 代表が異なる場合、それを使いましょう
            naive_union(C, u, v)                # 成分を結合
    return T

In [9]:
G = {
    0: {1: 1, 2: 3, 3: 4},
    1: {2: 5},
    2: {3: 2},
    3: set()
}
list(naive_kruskal(G))
# [(0, 1), (2, 3), (0, 2)]

[(0, 1), (2, 3), (0, 2)]

## リスト7-4 Kruskalのアルゴリズム


改善された`find`と`union`の実装は以下を参照してください。

In [10]:
def find(C, u):
    if C[u] != u:
        C[u] = find(C, C[u])                    # パス圧縮
    return C[u]


def union(C, R, u, v):
    u, v = find(C, u), find(C, v)
    if R[u] > R[v]:                             # ランクによる統合
        C[v] = u
    else:
        C[u] = v
    if R[u] == R[v]:                            # 同ランクの場合、vを1つ上にあげる
        R[v] += 1


def kruskal(G):
    E = [(G[u][v], u, v) for u in G for v in G[u]]
    T = set()
    C, R = {u: u for u in G}, {u: 0 for u in G}   # 成分の代表とランク
    for _, u, v in sorted(E):
        if find(C, u) != find(C, v):
            T.add((u, v))
            union(C, R, u, v)
    return T

In [11]:
G = {
    0: {1: 1, 2: 3, 3: 4},
    1: {2: 5},
    2: {3: 2},
    3: set()
}
list(kruskal(G))
# [(0, 1), (2, 3), (0, 2)]

[(0, 1), (2, 3), (0, 2)]

## Primのアルゴリズム

## リスト7-5 Primのアルゴリズム

以下に、Primのアルゴリズムの実装例を示します。

In [12]:
from heapq import heappop, heappush


def prim(G, s):
    P, Q = {}, [(0, None, s)]
    while Q:
        _, p, u = heappop(Q)
        if u in P:
            continue
        P[u] = p
        for v, w in G[u].items():
            heappush(Q, (w, u, v))
    return P

In [13]:
G = {
    0: {1: 1, 2: 3, 3: 4},
    1: {0: 1, 2: 5},
    2: {0: 3, 1: 5, 3: 2},
    3: {2: 2, 0: 4}
}
prim(G, 0)
# {0: None, 1: 0, 2: 0, 3: 2}

{0: None, 1: 0, 2: 0, 3: 2}